In [1]:
# Do transformers have an inductive bias towards high mutual information distributions?
#import os
#os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
import math
import numpy as np

"""
# Check if MPS is available and set the device accordingly
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Apple Silicon GPU)")
else:
    device = torch.device("cpu")
    print("MPS not available, using CPU")
"""
device=torch.device("cpu")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Model Hyperparameters
D_MODEL = 64       # Embedding dimension
N_HEAD = 8       # Number of attention heads
NUM_ENCODER_LAYERS = 3 # Number of encoder layers

# Training Hyperparameters
EPOCHS = 2000
LEARNING_RATE = 0.001
BATCH_SIZE = 32

In [3]:
def create_data():
    """Generates all possible two-digit addition problems."""
    data = []
    for i in range(101):
        for j in range(101):
            problem = f"{i}+{j}"
            solution = i+j
            data.append((problem, solution))
    return data

In [4]:
# Tokenization

# Define special symbols and vocabulary
SOS_TOKEN, EOS_TOKEN,PAD_TOKEN = 0,1,2
SRC_VOCAB = {str(i): i+3 for i in range(10)}
SRC_VOCAB['+'] = 13
# Inverse mapping for easy lookup
# Note: For this problem, the target is a class index (0-18), not a token sequence.
TGT_VOCAB_SIZE = 201 # Possible sums

def tokenize_problem(problem_str):
    """Converts a string like '3+5' to a list of token IDs."""
    tokens = []
    for char in problem_str:
        tokens.append(SRC_VOCAB[char])
    return tokens

In [5]:
# Make torch dataset

class AdditionDataset(Dataset):
    """Custom PyTorch Dataset for the addition task."""
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        problem, solution = self.data[idx]
        tokenized_problem = torch.tensor(tokenize_problem(problem), dtype=torch.long)
        return tokenized_problem, torch.tensor(solution, dtype=torch.long)

In [6]:
# define padding function
def collate_fn(batch):
    """Pads sequences in a batch to the same length."""
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(src_sample)
        tgt_batch.append(tgt_sample)

    # Pad source sequences
    src_padded = nn.utils.rnn.pad_sequence(src_batch, batch_first=True, padding_value=PAD_TOKEN)
    # Stack targets
    tgt_stacked = torch.stack(tgt_batch)
    return src_padded, tgt_stacked

In [7]:
# Transformer module
class PositionalEncoding(nn.Module):
    """Injects positional information into the input embeddings."""
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return x

class AdditionTransformer(nn.Module):
    """A Transformer model for the addition task (Encoder-only)."""
    def __init__(self, src_vocab_size, d_model, n_head, num_encoder_layers):
        super(AdditionTransformer, self).__init__()
        self.embedding = nn.Embedding(src_vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model, n_head, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        self.d_model = d_model
        # The output layer maps the aggregated encoder output to the 19 possible sum classes
        self.classifier = nn.Linear(d_model, TGT_VOCAB_SIZE)

    def forward(self, src, src_padding_mask):
        src_emb = self.embedding(src) * math.sqrt(self.d_model)
        src_emb = self.pos_encoder(src_emb.transpose(0, 1)).transpose(0, 1)
        
        # Pass through the encoder
        memory = self.transformer_encoder(src_emb, src_key_padding_mask=src_padding_mask)
        
        # We use the output of the [SOS] token for classification
        # The [SOS] token is always at the first position (index 0)
        output = memory[:, 0, :]
        
        # Final classification
        logits = self.classifier(output)
        return logits


In [8]:
# Training loop
def train_model(model, data_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for src, tgt in data_loader:
        src = src.to(device)
        tgt = tgt.to(device)
        optimizer.zero_grad()
        # Create a padding mask for the source
        src_padding_mask = (src == PAD_TOKEN)
        # Forward pass
        logits = model(src, src_padding_mask)
        # Calculate loss
        loss = criterion(logits, tgt)
        # Backpropagation
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(data_loader)

In [9]:
# prediction function
def predict(model, problem_str):
    """
    Takes a trained model and a problem string, returns the predicted answer
    and the probability distribution.
    """
    model.eval()
    with torch.no_grad():
        tokens = torch.tensor([tokenize_problem(problem_str)], dtype=torch.long, device=device)
        src_padding_mask = (tokens == PAD_TOKEN)
        
        logits = model(tokens, src_padding_mask)
        
        # Convert logits to probabilities using softmax
        probabilities = torch.softmax(logits, dim=1)
        
        # Get the predicted class (the one with the highest probability)
        predicted_index = torch.argmax(probabilities, dim=1).item()
        
        return predicted_index, probabilities.squeeze().cpu().numpy()

In [12]:
# execution

# Setup
print("--- 1. Setting up data ---")
all_data = create_data()
random.shuffle(all_data)

# All data used for training and testing
train_data = all_data[:8000]
test_data = all_data[2000:]

train_dataset = AdditionDataset(train_data)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

# Model Initialization
# Full vocab size includes PAD, SOS, EOS tokens
full_src_vocab_size = len(SRC_VOCAB) + 3 
model = AdditionTransformer(full_src_vocab_size, D_MODEL, N_HEAD, NUM_ENCODER_LAYERS).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0)

# Training
test_accuracies = []
losses = []
mean_entropies = []
print("\n--- 2. Starting training ---")
for epoch in range(1, EPOCHS + 1):
    optimizer.zero_grad()
    loss = train_model(model, train_dataloader, optimizer, criterion)
    if epoch % 10 == 0:
        correct_predictions = 0
        entropies = []
        for problem, solution in test_data:
            prediction, probs = predict(model, problem)
            entropies.append(-(probs * np.log(probs)).sum())
            if prediction == solution:
                correct_predictions += 1

        accuracy = (correct_predictions / len(test_data)) * 100
        test_accuracies.append(accuracy)
        losses.append(loss)
        mean_entropies.append(np.mean(entropies))
        print(f"Epoch: {epoch:02}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}%, Entropy: {np.mean(entropies):.4f}")
        
print("--- Training finished ---")

--- 1. Setting up data ---

--- 2. Starting training ---
Epoch: 10, Loss: 2.9992, Accuracy: 23.8629%, Entropy: 3.0450
Epoch: 20, Loss: 2.3156, Accuracy: 34.7397%, Entropy: 2.2809
Epoch: 30, Loss: 2.0417, Accuracy: 41.3242%, Entropy: 1.9614
Epoch: 40, Loss: 1.8611, Accuracy: 42.8972%, Entropy: 1.7311
Epoch: 50, Loss: 1.7582, Accuracy: 43.0070%, Entropy: 1.6138
Epoch: 60, Loss: 1.6799, Accuracy: 44.2263%, Entropy: 1.4758
Epoch: 70, Loss: 1.5910, Accuracy: 45.2140%, Entropy: 1.4006
Epoch: 80, Loss: 1.5342, Accuracy: 45.4213%, Entropy: 1.3254
Epoch: 90, Loss: 1.4716, Accuracy: 47.4576%, Entropy: 1.2732
Epoch: 100, Loss: 1.4322, Accuracy: 47.2869%, Entropy: 1.2540
Epoch: 110, Loss: 1.3562, Accuracy: 48.7623%, Entropy: 1.1811
Epoch: 120, Loss: 1.3589, Accuracy: 49.3598%, Entropy: 1.1654
Epoch: 130, Loss: 1.3544, Accuracy: 47.5796%, Entropy: 1.1418
Epoch: 140, Loss: 1.3120, Accuracy: 49.5793%, Entropy: 1.0686
Epoch: 150, Loss: 1.2672, Accuracy: 50.1524%, Entropy: 1.0118
Epoch: 160, Loss: 1.23

Epoch: 1320, Loss: 0.0659, Accuracy: 99.7561%, Entropy: 0.0094
Epoch: 1330, Loss: 0.0576, Accuracy: 99.8171%, Entropy: 0.0071
Epoch: 1340, Loss: 0.0555, Accuracy: 99.6098%, Entropy: 0.0097
Epoch: 1350, Loss: 0.0475, Accuracy: 99.4513%, Entropy: 0.0092
Epoch: 1360, Loss: 0.0516, Accuracy: 99.4635%, Entropy: 0.0086
Epoch: 1370, Loss: 0.0564, Accuracy: 99.7195%, Entropy: 0.0081
Epoch: 1380, Loss: 0.0503, Accuracy: 99.7561%, Entropy: 0.0068
Epoch: 1390, Loss: 0.0854, Accuracy: 99.5366%, Entropy: 0.0126
Epoch: 1400, Loss: 0.0478, Accuracy: 99.7439%, Entropy: 0.0054
Epoch: 1410, Loss: 0.0487, Accuracy: 99.6342%, Entropy: 0.0081
Epoch: 1420, Loss: 0.0584, Accuracy: 99.7074%, Entropy: 0.0086
Epoch: 1430, Loss: 0.0564, Accuracy: 99.4635%, Entropy: 0.0099
Epoch: 1440, Loss: 0.0519, Accuracy: 99.7439%, Entropy: 0.0069
Epoch: 1450, Loss: 0.0507, Accuracy: 99.6342%, Entropy: 0.0063
Epoch: 1460, Loss: 0.0818, Accuracy: 99.5732%, Entropy: 0.0095
Epoch: 1470, Loss: 0.0457, Accuracy: 99.8171%, Entropy:

In [13]:
# Evaluation
print("\n--- 3. Evaluating on test data ---")
correct_predictions = 0
entropies = []
for problem, solution in test_data:
    prediction, probs = predict(model, problem)
    entropies.append(-(probs * np.log(probs)).sum())
    is_correct = "Correct" if prediction == solution else f"Incorrect (expected {solution})"
    print(f"Problem: {problem}, Predicted: {prediction}  {is_correct}")
    if prediction == solution:
        correct_predictions += 1

accuracy = (correct_predictions / len(test_data)) * 100
print(f"\nTest Accuracy: {accuracy:.2f}%")

print(f"Entropy: {np.mean(entropies)}")


--- 3. Evaluating on test data ---
Problem: 65+17, Predicted: 82  Correct
Problem: 18+9, Predicted: 27  Correct
Problem: 0+28, Predicted: 28  Correct
Problem: 100+45, Predicted: 145  Correct
Problem: 63+30, Predicted: 93  Correct
Problem: 54+52, Predicted: 106  Correct
Problem: 58+11, Predicted: 69  Correct
Problem: 75+66, Predicted: 141  Correct
Problem: 29+18, Predicted: 47  Correct
Problem: 9+61, Predicted: 70  Correct
Problem: 74+21, Predicted: 95  Correct
Problem: 52+23, Predicted: 75  Correct
Problem: 92+92, Predicted: 184  Correct
Problem: 35+66, Predicted: 101  Correct
Problem: 29+88, Predicted: 117  Correct
Problem: 74+5, Predicted: 79  Correct
Problem: 72+49, Predicted: 121  Correct
Problem: 41+3, Predicted: 44  Correct
Problem: 82+28, Predicted: 110  Correct
Problem: 60+47, Predicted: 107  Correct
Problem: 43+67, Predicted: 110  Correct
Problem: 92+67, Predicted: 159  Correct
Problem: 17+57, Predicted: 74  Correct
Problem: 47+90, Predicted: 137  Correct
Problem: 43+22, Pred

Problem: 56+9, Predicted: 65  Correct
Problem: 44+20, Predicted: 64  Correct
Problem: 25+68, Predicted: 93  Correct
Problem: 32+39, Predicted: 71  Correct
Problem: 93+64, Predicted: 157  Correct
Problem: 22+26, Predicted: 48  Correct
Problem: 53+63, Predicted: 116  Correct
Problem: 82+45, Predicted: 127  Correct
Problem: 25+45, Predicted: 70  Correct
Problem: 80+67, Predicted: 147  Correct
Problem: 30+70, Predicted: 100  Correct
Problem: 75+55, Predicted: 130  Correct
Problem: 1+66, Predicted: 67  Correct
Problem: 2+41, Predicted: 43  Correct
Problem: 52+71, Predicted: 123  Correct
Problem: 4+68, Predicted: 72  Correct
Problem: 56+41, Predicted: 97  Correct
Problem: 98+78, Predicted: 176  Correct
Problem: 38+97, Predicted: 135  Correct
Problem: 40+21, Predicted: 61  Correct
Problem: 38+41, Predicted: 79  Correct
Problem: 13+84, Predicted: 97  Correct
Problem: 42+65, Predicted: 107  Correct
Problem: 54+33, Predicted: 87  Correct
Problem: 7+2, Predicted: 9  Correct
Problem: 66+19, Predic

Problem: 7+61, Predicted: 68  Correct
Problem: 33+3, Predicted: 36  Correct
Problem: 46+90, Predicted: 136  Correct
Problem: 85+62, Predicted: 147  Correct
Problem: 25+4, Predicted: 29  Correct
Problem: 59+30, Predicted: 89  Correct
Problem: 20+7, Predicted: 27  Correct
Problem: 22+77, Predicted: 99  Correct
Problem: 55+1, Predicted: 56  Correct
Problem: 95+82, Predicted: 177  Correct
Problem: 39+39, Predicted: 78  Correct
Problem: 41+4, Predicted: 45  Correct
Problem: 62+51, Predicted: 113  Correct
Problem: 28+92, Predicted: 120  Correct
Problem: 24+22, Predicted: 46  Correct
Problem: 58+87, Predicted: 145  Correct
Problem: 82+31, Predicted: 113  Correct
Problem: 46+15, Predicted: 61  Correct
Problem: 26+77, Predicted: 103  Correct
Problem: 50+49, Predicted: 99  Correct
Problem: 86+60, Predicted: 146  Correct
Problem: 45+28, Predicted: 73  Correct
Problem: 97+52, Predicted: 149  Correct
Problem: 100+28, Predicted: 128  Correct
Problem: 25+43, Predicted: 68  Correct
Problem: 82+96, Pre

Problem: 63+96, Predicted: 159  Correct
Problem: 77+87, Predicted: 164  Correct
Problem: 34+61, Predicted: 95  Correct
Problem: 15+39, Predicted: 54  Correct
Problem: 64+40, Predicted: 104  Correct
Problem: 26+26, Predicted: 52  Correct
Problem: 21+58, Predicted: 79  Correct
Problem: 40+13, Predicted: 53  Correct
Problem: 88+9, Predicted: 97  Correct
Problem: 35+39, Predicted: 74  Correct
Problem: 65+18, Predicted: 83  Correct
Problem: 20+23, Predicted: 43  Correct
Problem: 20+41, Predicted: 61  Correct
Problem: 58+24, Predicted: 82  Correct
Problem: 30+82, Predicted: 112  Correct
Problem: 83+19, Predicted: 102  Correct
Problem: 44+93, Predicted: 137  Correct
Problem: 61+78, Predicted: 139  Correct
Problem: 25+12, Predicted: 37  Correct
Problem: 22+24, Predicted: 46  Correct
Problem: 70+13, Predicted: 83  Correct
Problem: 15+14, Predicted: 29  Correct
Problem: 68+95, Predicted: 163  Correct
Problem: 24+89, Predicted: 113  Correct
Problem: 48+91, Predicted: 139  Correct
Problem: 80+11, 

Problem: 97+79, Predicted: 176  Correct
Problem: 23+67, Predicted: 90  Correct
Problem: 81+9, Predicted: 90  Correct
Problem: 93+45, Predicted: 138  Correct
Problem: 60+1, Predicted: 61  Correct
Problem: 14+51, Predicted: 65  Correct
Problem: 31+73, Predicted: 104  Correct
Problem: 85+92, Predicted: 177  Correct
Problem: 39+70, Predicted: 109  Correct
Problem: 65+99, Predicted: 164  Correct
Problem: 43+81, Predicted: 124  Correct
Problem: 96+50, Predicted: 146  Correct
Problem: 61+41, Predicted: 102  Correct
Problem: 58+42, Predicted: 100  Correct
Problem: 81+88, Predicted: 169  Correct
Problem: 95+56, Predicted: 151  Correct
Problem: 8+28, Predicted: 36  Correct
Problem: 86+76, Predicted: 162  Correct
Problem: 6+0, Predicted: 6  Correct
Problem: 99+18, Predicted: 117  Correct
Problem: 74+73, Predicted: 147  Correct
Problem: 86+32, Predicted: 118  Correct
Problem: 6+87, Predicted: 93  Correct
Problem: 0+87, Predicted: 87  Correct
Problem: 92+27, Predicted: 119  Correct
Problem: 17+39, 

...

Problem: 7+98, Predicted: 105  Correct
Problem: 94+43, Predicted: 137  Correct
Problem: 47+60, Predicted: 107  Correct
Problem: 35+99, Predicted: 134  Correct
Problem: 8+79, Predicted: 87  Correct
Problem: 0+82, Predicted: 82  Correct
Problem: 68+34, Predicted: 102  Correct
Problem: 75+28, Predicted: 103  Correct
Problem: 82+79, Predicted: 161  Correct
Problem: 10+83, Predicted: 93  Correct
Problem: 62+12, Predicted: 74  Correct
Problem: 54+83, Predicted: 137  Correct
Problem: 81+78, Predicted: 159  Correct
Problem: 55+14, Predicted: 69  Correct
Problem: 83+100, Predicted: 183  Correct
Problem: 10+100, Predicted: 110  Correct
Problem: 87+86, Predicted: 173  Correct
Problem: 44+6, Predicted: 50  Correct
Problem: 83+99, Predicted: 182  Correct
Problem: 27+47, Predicted: 74  Correct
Problem: 90+41, Predicted: 131  Correct
Problem: 36+3, Predicted: 39  Correct
Problem: 90+7, Predicted: 97  Correct
Problem: 23+73, Predicted: 96  Correct
Problem: 46+51, Predicted: 97  Correct
Problem: 10+55,

Problem: 40+39, Predicted: 79  Correct
Problem: 67+41, Predicted: 108  Correct
Problem: 72+11, Predicted: 83  Correct
Problem: 3+98, Predicted: 101  Correct
Problem: 15+26, Predicted: 41  Correct
Problem: 99+94, Predicted: 193  Correct
Problem: 43+29, Predicted: 72  Correct
Problem: 54+62, Predicted: 116  Correct
Problem: 8+80, Predicted: 88  Correct
Problem: 20+3, Predicted: 23  Correct
Problem: 64+100, Predicted: 164  Correct
Problem: 61+0, Predicted: 61  Correct
Problem: 15+23, Predicted: 38  Correct
Problem: 8+69, Predicted: 77  Correct
Problem: 31+2, Predicted: 33  Correct
Problem: 84+36, Predicted: 120  Correct
Problem: 40+48, Predicted: 88  Correct
Problem: 46+30, Predicted: 76  Correct
Problem: 62+50, Predicted: 112  Correct
Problem: 26+63, Predicted: 89  Correct
Problem: 39+83, Predicted: 122  Correct
Problem: 41+79, Predicted: 120  Correct
Problem: 98+17, Predicted: 115  Correct
Problem: 33+44, Predicted: 77  Correct
Problem: 28+37, Predicted: 65  Correct
Problem: 94+21, Pred

Problem: 10+39, Predicted: 49  Correct
Problem: 70+87, Predicted: 157  Correct
Problem: 47+37, Predicted: 84  Correct
Problem: 13+72, Predicted: 85  Correct
Problem: 21+55, Predicted: 76  Correct
Problem: 87+79, Predicted: 166  Correct
Problem: 2+22, Predicted: 24  Correct
Problem: 32+58, Predicted: 90  Correct
Problem: 31+77, Predicted: 108  Correct
Problem: 87+40, Predicted: 127  Correct
Problem: 5+32, Predicted: 37  Correct
Problem: 74+57, Predicted: 131  Correct
Problem: 26+23, Predicted: 49  Correct
Problem: 36+28, Predicted: 64  Correct
Problem: 11+4, Predicted: 15  Correct
Problem: 52+66, Predicted: 118  Correct
Problem: 48+65, Predicted: 113  Correct
Problem: 63+9, Predicted: 72  Correct
Problem: 80+28, Predicted: 108  Correct
Problem: 50+50, Predicted: 100  Correct
Problem: 81+71, Predicted: 152  Correct
Problem: 3+56, Predicted: 59  Correct
Problem: 49+59, Predicted: 108  Correct
Problem: 63+57, Predicted: 120  Correct
Problem: 90+94, Predicted: 184  Correct
Problem: 79+16, P

Problem: 25+87, Predicted: 112  Correct
Problem: 19+51, Predicted: 70  Correct
Problem: 79+14, Predicted: 93  Correct
Problem: 90+44, Predicted: 134  Correct
Problem: 28+70, Predicted: 98  Correct
Problem: 11+77, Predicted: 88  Correct
Problem: 22+90, Predicted: 112  Correct
Problem: 37+12, Predicted: 49  Correct
Problem: 15+97, Predicted: 112  Correct
Problem: 7+25, Predicted: 32  Correct
Problem: 68+84, Predicted: 152  Correct
Problem: 51+84, Predicted: 135  Correct
Problem: 54+32, Predicted: 86  Correct
Problem: 69+12, Predicted: 81  Correct
Problem: 73+58, Predicted: 131  Correct
Problem: 44+36, Predicted: 80  Correct
Problem: 16+85, Predicted: 101  Correct
Problem: 70+77, Predicted: 147  Correct
Problem: 74+64, Predicted: 138  Correct
Problem: 47+38, Predicted: 85  Correct
Problem: 39+61, Predicted: 100  Correct
Problem: 44+45, Predicted: 89  Correct
Problem: 23+43, Predicted: 66  Correct
Problem: 18+77, Predicted: 95  Correct
Problem: 75+30, Predicted: 105  Correct
Problem: 89+56

Problem: 28+83, Predicted: 111  Correct
Problem: 73+18, Predicted: 91  Correct
Problem: 27+5, Predicted: 32  Correct
Problem: 36+68, Predicted: 104  Correct
Problem: 26+15, Predicted: 41  Correct
Problem: 47+97, Predicted: 144  Correct
Problem: 72+59, Predicted: 131  Correct
Problem: 7+44, Predicted: 51  Correct
Problem: 23+76, Predicted: 99  Correct
Problem: 76+84, Predicted: 160  Correct
Problem: 83+9, Predicted: 92  Correct
Problem: 25+59, Predicted: 84  Correct
Problem: 15+100, Predicted: 115  Correct
Problem: 13+18, Predicted: 31  Correct
Problem: 8+82, Predicted: 90  Correct
Problem: 7+38, Predicted: 45  Correct
Problem: 53+78, Predicted: 131  Correct
Problem: 48+95, Predicted: 143  Correct
Problem: 85+16, Predicted: 101  Correct
Problem: 3+64, Predicted: 67  Correct
Problem: 36+70, Predicted: 106  Correct
Problem: 83+0, Predicted: 83  Correct
Problem: 22+70, Predicted: 92  Correct
Problem: 18+47, Predicted: 65  Correct
Problem: 62+28, Predicted: 90  Correct
Problem: 95+83, Predi

In [14]:
np.log(TGT_VOCAB_SIZE)

5.303304908059076

In [15]:
# save weights in state dict
torch.save(model.state_dict(), "addition_model_state_dict.pth")

# save model in pckl
torch.save(model, "addition_model.pckl")

In [16]:
# example loading the model
#full_src_vocab_size = len(SRC_VOCAB) + 3
#model= AdditionTransformer(full_src_vocab_size, D_MODEL, N_HEAD, NUM_ENCODER_LAYERS)
#state_dict = torch.load("addition_model_state_dict.pth", weights_only=True)
#model.load_state_dict(state_dict)

In [17]:
# saving data
import pickle
with open('addition_data.pkl', 'wb') as f:
    pickle.dump(all_data, f)
    
with open('test_accuracies_addition.pkl', 'wb') as f:
    pickle.dump(test_accuracies, f)
    
with open('losses_addition.pkl', 'wb') as f:
    pickle.dump(losses, f)

with open('mean_entropies_addition.pkl', 'wb') as f:
    pickle.dump(mean_entropies, f)

# Random data

In [18]:
def create_data_random():
    """Generates all possible two-digit addition problems."""
    data = []
    for i in range(101):
        for j in range(101):
            problem = f"{i}+{j}"
            solution = np.random.randint(low=0,high=200)
            data.append((problem, solution))
    return data

In [19]:
# execution

# Setup
print("--- 1. Setting up data ---")
all_data_random = create_data_random()
random.shuffle(all_data_random)

# All data used for training and testing
train_data = all_data_random[:8000]
test_data = all_data_random[8000:]

train_dataset = AdditionDataset(train_data)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

# Model Initialization
# Full vocab size includes PAD, SOS, EOS tokens
full_src_vocab_size = len(SRC_VOCAB) + 3 
random_model = AdditionTransformer(full_src_vocab_size, D_MODEL, N_HEAD, NUM_ENCODER_LAYERS).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(random_model.parameters(), lr=LEARNING_RATE)

# Training
test_accuracies = []
losses = []
mean_entropies = []
print("\n--- 2. Starting training ---")
for epoch in range(1, EPOCHS + 1):
    optimizer.zero_grad()
    loss = train_model(random_model, train_dataloader, optimizer, criterion)
    if epoch % 10 == 0:
        correct_predictions = 0
        entropies = []
        for problem, solution in test_data:
            prediction, probs = predict(random_model, problem)
            entropies.append(-(probs * np.log(probs)).sum())
            if prediction == solution:
                correct_predictions += 1

        accuracy = (correct_predictions / len(test_data)) * 100
        test_accuracies.append(accuracy)
        losses.append(loss)
        mean_entropies.append(np.mean(entropies))
        print(f"Epoch: {epoch:02}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}%, Entropy: {np.mean(entropies):.4f}")
        
print("--- Training finished ---")

--- 1. Setting up data ---

--- 2. Starting training ---
Epoch: 10, Loss: 5.2348, Accuracy: 0.4998%, Entropy: 5.2148
Epoch: 20, Loss: 4.9260, Accuracy: 0.8632%, Entropy: 4.9724
Epoch: 30, Loss: 4.4890, Accuracy: 0.5452%, Entropy: 4.5838
Epoch: 40, Loss: 4.0962, Accuracy: 0.4998%, Entropy: 4.1599
Epoch: 50, Loss: 3.7673, Accuracy: 0.4089%, Entropy: 3.7665
Epoch: 60, Loss: 3.5504, Accuracy: 0.4089%, Entropy: 3.4575
Epoch: 70, Loss: 3.3723, Accuracy: 0.3180%, Entropy: 3.2277
Epoch: 80, Loss: 3.2873, Accuracy: 0.4543%, Entropy: 3.0762
Epoch: 90, Loss: 3.1656, Accuracy: 0.4089%, Entropy: 2.9304
Epoch: 100, Loss: 3.0541, Accuracy: 0.4543%, Entropy: 2.8078
Epoch: 110, Loss: 2.9263, Accuracy: 0.5452%, Entropy: 2.6317
Epoch: 120, Loss: 2.8406, Accuracy: 0.6361%, Entropy: 2.5058
Epoch: 130, Loss: 2.7838, Accuracy: 0.5452%, Entropy: 2.3683
Epoch: 140, Loss: 2.6830, Accuracy: 0.7269%, Entropy: 2.2699
Epoch: 150, Loss: 2.6144, Accuracy: 0.4543%, Entropy: 2.1543
Epoch: 160, Loss: 2.5635, Accuracy: 0

Epoch: 1340, Loss: 0.3073, Accuracy: 0.5906%, Entropy: 1.0527
Epoch: 1350, Loss: 0.3249, Accuracy: 0.2726%, Entropy: 1.0783
Epoch: 1360, Loss: 0.3079, Accuracy: 0.3180%, Entropy: 1.0850
Epoch: 1370, Loss: 0.3255, Accuracy: 0.4998%, Entropy: 1.0787
Epoch: 1380, Loss: 0.3214, Accuracy: 0.4543%, Entropy: 1.0865
Epoch: 1390, Loss: 0.3254, Accuracy: 0.4998%, Entropy: 1.0547
Epoch: 1400, Loss: 0.3100, Accuracy: 0.2726%, Entropy: 1.0823
Epoch: 1410, Loss: 0.3079, Accuracy: 0.3180%, Entropy: 1.0879
Epoch: 1420, Loss: 0.3035, Accuracy: 0.4998%, Entropy: 1.0673
Epoch: 1430, Loss: 0.3017, Accuracy: 0.5452%, Entropy: 1.0656
Epoch: 1440, Loss: 0.3273, Accuracy: 0.4998%, Entropy: 1.0859
Epoch: 1450, Loss: 0.3119, Accuracy: 0.4543%, Entropy: 1.0573
Epoch: 1460, Loss: 0.2831, Accuracy: 0.1817%, Entropy: 1.0811
Epoch: 1470, Loss: 0.2749, Accuracy: 0.4089%, Entropy: 1.0703
Epoch: 1480, Loss: 0.3160, Accuracy: 0.4543%, Entropy: 1.0813
Epoch: 1490, Loss: 0.2958, Accuracy: 0.3180%, Entropy: 1.0967
Epoch: 1

In [20]:
# Evaluation
print("\n--- 3. Evaluating on test data, should get random accuracy and low entropy ---")
correct_predictions = 0
entropies = []
for problem, solution in test_data:
    prediction, probs = predict(random_model, problem)
    entropies.append(-(probs * np.log(probs)).sum())
    is_correct = "Correct" if prediction == solution else f"Incorrect (expected {solution})"
    print(f"Problem: {problem}, Predicted: {prediction}  {is_correct}")
    if prediction == solution:
        correct_predictions += 1

accuracy = (correct_predictions / len(test_data)) * 100
print(f"\nTest Accuracy: {accuracy:.2f}%")

print(f"Entropy: {np.mean(entropies)}")


--- 3. Evaluating on test data, should get random accuracy and low entropy ---
Problem: 88+82, Predicted: 63  Incorrect (expected 171)
Problem: 3+54, Predicted: 119  Incorrect (expected 138)
Problem: 19+97, Predicted: 126  Incorrect (expected 92)
Problem: 78+4, Predicted: 113  Incorrect (expected 128)
Problem: 93+49, Predicted: 174  Incorrect (expected 100)
Problem: 96+43, Predicted: 44  Incorrect (expected 9)
Problem: 58+86, Predicted: 100  Incorrect (expected 143)
Problem: 14+82, Predicted: 35  Incorrect (expected 7)
Problem: 0+27, Predicted: 131  Incorrect (expected 7)
Problem: 30+61, Predicted: 153  Incorrect (expected 175)
Problem: 44+99, Predicted: 57  Incorrect (expected 45)
Problem: 93+21, Predicted: 183  Incorrect (expected 132)
Problem: 17+99, Predicted: 139  Incorrect (expected 0)
Problem: 35+91, Predicted: 8  Incorrect (expected 39)
Problem: 60+27, Predicted: 98  Incorrect (expected 91)
Problem: 57+54, Predicted: 85  Incorrect (expected 4)
Problem: 23+81, Predicted: 119  I

Problem: 76+41, Predicted: 35  Incorrect (expected 77)
Problem: 99+96, Predicted: 170  Incorrect (expected 0)
Problem: 93+83, Predicted: 18  Incorrect (expected 81)
Problem: 92+78, Predicted: 153  Incorrect (expected 154)
Problem: 62+48, Predicted: 127  Incorrect (expected 134)
Problem: 73+0, Predicted: 30  Incorrect (expected 67)
Problem: 13+14, Predicted: 83  Incorrect (expected 24)
Problem: 54+14, Predicted: 188  Incorrect (expected 155)
Problem: 83+38, Predicted: 51  Incorrect (expected 83)
Problem: 21+73, Predicted: 24  Incorrect (expected 172)
Problem: 42+94, Predicted: 59  Incorrect (expected 94)
Problem: 98+95, Predicted: 102  Incorrect (expected 93)
Problem: 60+12, Predicted: 126  Incorrect (expected 80)
Problem: 30+69, Predicted: 73  Incorrect (expected 108)
Problem: 91+79, Predicted: 6  Incorrect (expected 100)
Problem: 33+28, Predicted: 57  Incorrect (expected 176)
Problem: 99+34, Predicted: 174  Incorrect (expected 125)
Problem: 78+11, Predicted: 105  Incorrect (expected 4

Problem: 54+64, Predicted: 23  Incorrect (expected 44)
Problem: 51+9, Predicted: 101  Incorrect (expected 28)
Problem: 78+50, Predicted: 155  Incorrect (expected 191)
Problem: 20+58, Predicted: 15  Incorrect (expected 168)
Problem: 2+25, Predicted: 178  Incorrect (expected 176)
Problem: 80+76, Predicted: 170  Incorrect (expected 89)
Problem: 74+82, Predicted: 158  Incorrect (expected 191)
Problem: 85+10, Predicted: 168  Incorrect (expected 175)
Problem: 34+0, Predicted: 153  Incorrect (expected 126)
Problem: 52+93, Predicted: 108  Incorrect (expected 156)
Problem: 22+94, Predicted: 129  Incorrect (expected 153)
Problem: 65+61, Predicted: 164  Incorrect (expected 93)
Problem: 60+51, Predicted: 110  Incorrect (expected 143)
Problem: 55+87, Predicted: 92  Incorrect (expected 64)
Problem: 53+19, Predicted: 70  Incorrect (expected 34)
Problem: 57+93, Predicted: 6  Incorrect (expected 71)
Problem: 43+57, Predicted: 167  Incorrect (expected 198)
Problem: 61+53, Predicted: 71  Incorrect (expec

Problem: 14+16, Predicted: 71  Incorrect (expected 135)
Problem: 29+52, Predicted: 63  Incorrect (expected 126)
Problem: 97+23, Predicted: 77  Incorrect (expected 4)
Problem: 24+60, Predicted: 137  Incorrect (expected 187)
Problem: 61+2, Predicted: 174  Incorrect (expected 30)
Problem: 73+85, Predicted: 103  Incorrect (expected 6)
Problem: 40+69, Predicted: 88  Incorrect (expected 189)
Problem: 67+68, Predicted: 165  Incorrect (expected 46)
Problem: 66+27, Predicted: 151  Incorrect (expected 98)
Problem: 100+13, Predicted: 134  Incorrect (expected 73)
Problem: 55+11, Predicted: 156  Incorrect (expected 139)
Problem: 7+50, Predicted: 153  Incorrect (expected 22)
Problem: 8+72, Predicted: 22  Incorrect (expected 93)
Problem: 73+59, Predicted: 27  Incorrect (expected 2)
Problem: 27+41, Predicted: 115  Incorrect (expected 44)
Problem: 7+18, Predicted: 171  Incorrect (expected 132)
Problem: 26+95, Predicted: 101  Incorrect (expected 49)
Problem: 89+37, Predicted: 82  Incorrect (expected 101

Problem: 5+60, Predicted: 114  Incorrect (expected 179)
Problem: 0+3, Predicted: 168  Incorrect (expected 31)
Problem: 47+99, Predicted: 87  Incorrect (expected 13)
Problem: 82+28, Predicted: 128  Incorrect (expected 40)
Problem: 75+22, Predicted: 11  Incorrect (expected 136)
Problem: 67+85, Predicted: 180  Incorrect (expected 20)
Problem: 35+9, Predicted: 83  Incorrect (expected 87)
Problem: 71+71, Predicted: 196  Incorrect (expected 36)
Problem: 20+9, Predicted: 186  Incorrect (expected 38)
Problem: 80+44, Predicted: 119  Incorrect (expected 184)
Problem: 83+100, Predicted: 104  Incorrect (expected 72)
Problem: 23+92, Predicted: 134  Incorrect (expected 136)
Problem: 48+82, Predicted: 79  Incorrect (expected 73)
Problem: 0+80, Predicted: 199  Incorrect (expected 9)
Problem: 85+90, Predicted: 120  Incorrect (expected 34)
Problem: 61+37, Predicted: 61  Incorrect (expected 183)
Problem: 46+23, Predicted: 160  Incorrect (expected 126)
Problem: 55+61, Predicted: 104  Incorrect (expected 4

Problem: 15+9, Predicted: 114  Incorrect (expected 12)
Problem: 1+43, Predicted: 158  Incorrect (expected 2)
Problem: 91+63, Predicted: 36  Incorrect (expected 59)
Problem: 13+51, Predicted: 135  Incorrect (expected 60)
Problem: 30+49, Predicted: 160  Incorrect (expected 90)
Problem: 68+83, Predicted: 23  Incorrect (expected 98)
Problem: 13+67, Predicted: 58  Incorrect (expected 7)
Problem: 7+19, Predicted: 158  Incorrect (expected 183)
Problem: 97+47, Predicted: 133  Incorrect (expected 80)
Problem: 88+86, Predicted: 183  Incorrect (expected 93)
Problem: 51+49, Predicted: 167  Incorrect (expected 55)
Problem: 75+59, Predicted: 27  Incorrect (expected 84)
Problem: 14+25, Predicted: 131  Incorrect (expected 140)
Problem: 81+57, Predicted: 64  Incorrect (expected 70)
Problem: 27+54, Predicted: 160  Incorrect (expected 136)
Problem: 81+36, Predicted: 68  Incorrect (expected 75)
Problem: 98+32, Predicted: 124  Incorrect (expected 83)
Problem: 34+98, Predicted: 125  Incorrect (expected 121)

Problem: 71+28, Predicted: 64  Incorrect (expected 155)
Problem: 58+76, Predicted: 106  Incorrect (expected 116)
Problem: 21+98, Predicted: 80  Incorrect (expected 131)
Problem: 81+67, Predicted: 169  Incorrect (expected 146)
Problem: 36+39, Predicted: 125  Incorrect (expected 82)
Problem: 25+33, Predicted: 91  Incorrect (expected 26)
Problem: 56+48, Predicted: 65  Incorrect (expected 71)
Problem: 15+15, Predicted: 154  Incorrect (expected 93)
Problem: 93+13, Predicted: 44  Incorrect (expected 172)
Problem: 65+32, Predicted: 58  Incorrect (expected 33)
Problem: 90+9, Predicted: 64  Incorrect (expected 165)
Problem: 14+58, Predicted: 113  Incorrect (expected 100)
Problem: 16+71, Predicted: 98  Incorrect (expected 1)
Problem: 25+54, Predicted: 160  Incorrect (expected 31)
Problem: 34+70, Predicted: 68  Incorrect (expected 4)
Problem: 73+97, Predicted: 43  Incorrect (expected 133)
Problem: 4+44, Predicted: 160  Incorrect (expected 168)
Problem: 6+10, Predicted: 90  Incorrect (expected 173

Problem: 86+60, Predicted: 26  Incorrect (expected 43)
Problem: 88+31, Predicted: 104  Incorrect (expected 139)
Problem: 2+100, Predicted: 142  Incorrect (expected 3)
Problem: 9+97, Predicted: 71  Incorrect (expected 89)
Problem: 81+82, Predicted: 105  Incorrect (expected 174)
Problem: 31+71, Predicted: 0  Incorrect (expected 74)
Problem: 14+95, Predicted: 100  Incorrect (expected 188)
Problem: 59+43, Predicted: 148  Incorrect (expected 105)
Problem: 42+82, Predicted: 76  Incorrect (expected 62)
Problem: 91+92, Predicted: 192  Incorrect (expected 170)
Problem: 66+55, Predicted: 27  Incorrect (expected 58)
Problem: 91+14, Predicted: 151  Incorrect (expected 133)
Problem: 6+99, Predicted: 169  Incorrect (expected 189)
Problem: 1+13, Predicted: 134  Incorrect (expected 191)
Problem: 57+31, Predicted: 45  Incorrect (expected 127)
Problem: 40+65, Predicted: 190  Incorrect (expected 195)
Problem: 86+43, Predicted: 96  Incorrect (expected 85)
Problem: 2+35, Predicted: 2  Incorrect (expected 1

Problem: 59+61, Predicted: 138  Incorrect (expected 182)
Problem: 64+78, Predicted: 100  Incorrect (expected 185)
Problem: 82+45, Predicted: 129  Incorrect (expected 158)
Problem: 32+55, Predicted: 98  Incorrect (expected 3)
Problem: 28+68, Predicted: 139  Incorrect (expected 117)
Problem: 25+42, Predicted: 10  Incorrect (expected 7)
Problem: 62+38, Predicted: 134  Incorrect (expected 182)
Problem: 71+72, Predicted: 133  Incorrect (expected 195)
Problem: 89+51, Predicted: 124  Incorrect (expected 30)
Problem: 1+66, Predicted: 74  Incorrect (expected 26)
Problem: 29+77, Predicted: 160  Incorrect (expected 161)
Problem: 24+2, Predicted: 35  Incorrect (expected 0)
Problem: 6+13, Predicted: 72  Incorrect (expected 9)
Problem: 54+88, Predicted: 102  Incorrect (expected 117)
Problem: 5+13, Predicted: 157  Incorrect (expected 133)
Problem: 72+7, Predicted: 162  Incorrect (expected 65)
Problem: 38+39, Predicted: 36  Incorrect (expected 70)
Problem: 27+82, Predicted: 133  Incorrect (expected 14

In [21]:
# save weights in state dict
torch.save(random_model.state_dict(), "random_model_state_dict.pth")

# save model in pckl
torch.save(random_model, "random_model.pckl")

In [22]:
# saving data
import pickle
with open('random_data.pkl', 'wb') as f:
    pickle.dump(all_data_random, f)
    
with open('test_accuracies_random.pkl', 'wb') as f:
    pickle.dump(test_accuracies, f)
    
with open('losses_random.pkl', 'wb') as f:
    pickle.dump(losses, f)

with open('mean_entropies_random.pkl', 'wb') as f:
    pickle.dump(mean_entropies, f)